# TCLab Help
## Automatically open and close TCLab
Opening and closing can cause problems if the file was not closed correctly. To do this safer, use a with statement
## Use time.time() to execute loop at the beginning of every second
time.time() returns the current time in seconds. The below code uses the difference in two time.time() instances to execute the code only when the difference is exactly an integer. Since the code within the loop should not take more than a second, the loop will execute once every second.

In [ ]:
import matplotlib.pyplot as plt
import tclab
import time

n = 300  # Number of second time points (5 min)
tm = np.linspace(0,n,n+1) # Time values

# data
with tclab.TCLab() as lab:
    T1_list = [lab.T1]
    lab.Q1(50)
    t_meas = [0]  # Record actual time
    t0 = time.time()
    for i in range(n):
        # wait
        time.sleep(1-time.time()%1)
        #measure and record
        t_meas.append(time.time() - t0)
        T1_list.append(lab.T1)

# Plot results
plt.figure(1)
plt.plot(t_meas,T1_list,'r.',label='Measured')
plt.ylabel('Temperature (degC)')
plt.xlabel('Time (sec)')
plt.legend()
plt.show()


## Use Error Handling, and run TCLab once
It can become tedious to run the TCLab each time you debug. To help, we can use error handling so that once the TCLab successfully runs, we save the data for later.

When Python encounters a problem, it "throws" an exception. You may recognize some of these: IndexError when a list or array does not have the index referenced, or SyntaxError for common syntax problems. With __try catch__ blocks, we can tell Python what we want it do do, and what to do if there is an exception

Below, the __try__ block will tell it to read the file we saved our TCLab data in. If it does not exist (for example, if this is the first time running it) then an exception will be thrown and the code will stop executing in th try block and move to the __except__ block. There, we will run the TCLab and save it for later retreival. This way, if we have problems after the TCLab portion, we will not have to wait for it to simulate again
## Read and save files with Pandas or numpy
### Numpy
To save data with numpy, we will use `numpy.savetxt`. The data should be formatted as an array first, then the syntax will be `numpy.savetxt([file path], [array], delimiter=",")` where `[array]` is the array of data being saved, and `[file path]` is the path to where you will save the data. The path will be saved in the same directory as the Python file if only a file name is included. For more details, visit [here](https://numpy.org/doc/stable/reference/generated/numpy.savetxt.html)

To load data from a CSV into an array, use `numpy.loadtxt`. This has similar syntax: `numpy.savetxt([file path], delimiter=",")` with many more options available [here](https://numpy.org/doc/stable/reference/generated/numpy.loadtxt.html)

In [ ]:
import matplotlib.pyplot as plt
import tclab
import time

filename = "TCLAb_data1.csv"
try:
    data = np.loadtxt(filename, delimiter=",")
    data = data.T  # transpose so that each row is all values of that type
    T_meas, T1_list = data  # unpack 2d array into 2 arrays we would have had if we ran the TCLab
    
except: 
    # Gather Data
    n = 300  # Number of second time points (5 min)
    tm = np.linspace(0,n,n+1) # Time values

    with tclab.TCLab() as lab:
        T1_list = [lab.T1]
        lab.Q1(50)
        t_meas = [0]  # Record actual time
        t0 = time.time()
        for i in range(n):
            time.sleep(1-time.time()%1)
            t_meas.append(time.time() - t0)
            T1_list.append(lab.T1)
            
    # save data
    data = np.array([t_meas, t1_list]).T  # .T transposes so that the data is arranged by collumns
    np.savetxt(filename, data, delimiter=",")

# Plot results
plt.figure(1)
plt.plot(t_meas,T1_list,'r.',label='Measured')
plt.ylabel('Temperature (degC)')
plt.xlabel('Time (sec)')
plt.legend()
plt.show()

## Only Run the TCLab if Cooled
Running the TCLab while still hot from a previous experiement can cause issues within the simulation. To prevent this, use a while loop to automatically wait for the TCLab to cool to a proper temperature.

In [ ]:
import tclab
import time

with tclab.TCLab() as lab:
    while lab.T1 > 30:  # wait for TCLab to be below 30 C before simulating
        time.sleep(5)
        print(f'TCLab is now {lab.T1} degrees C')
        print('Waiting to cool...')
    print(f'TCLab is now {lab_temp} degrees C')
    print('Ready to Start!')
    

In [ ]:
()